In [92]:
bikes = spark.read.option("header", "true").option("inferSchema", "true").csv("abfss://landing@mikkosynapse02adls.dfs.core.windows.net/hsl/citybikes/2023/")
display(bikes)
bikes.count()

StatementMeta(kipina, 10, 87, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b2609188-03d1-4c0d-9bb4-b7eae22653c0)

5088050

In [90]:
observations = spark.read.json("abfss://landing@mikkosynapse02adls.dfs.core.windows.net/fmi/weather/")
display(observations)

StatementMeta(kipina, 10, 85, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d6d34410-0901-403e-8f65-a39a442c71fc)

In [91]:
from pyspark.sql.functions import *

b = observations.select(inline(col("wfs:FeatureCollection.wfs:member"))).select("omso:PointTimeSeriesObservation.*")
temp_a = b.where(col("om:observedProperty.xlink:@href").like("%param=tmin%")).select("om:result.wml2:MeasurementTimeseries.wml2:point")
temp_b = temp_a.select(inline("wml2:point")).withColumn("ObservationTime", col("wml2:MeasurementTVP.wml2:time")).withColumn("Value", col("wml2:MeasurementTVP.wml2:value"))
temp_c = temp_b.withColumn("ObservationDate", to_date(col("ObservationTime")))
temp_observations = temp_c.select(["ObservationDate", "Value"]).where(~isnan(col("Value"))).orderBy(col("ObservationTime"))
display(temp_observations)


StatementMeta(kipina, 10, 86, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0a94e610-1e38-4865-967d-397ad08c8cc5)

In [97]:
rain_a = b.where(col("om:observedProperty.xlink:@href").like("%param=rrday%")).select("om:result.wml2:MeasurementTimeseries.wml2:point")
rain_b = rain_a.select(inline("wml2:point")).withColumn("ObservationTime", col("wml2:MeasurementTVP.wml2:time")).withColumn("Value", col("wml2:MeasurementTVP.wml2:value"))
rain_c = rain_b.withColumn("ObservationDate", to_date(col("ObservationTime")))
rain_observations = rain_c.select(["ObservationDate", "Value"]).where(~isnan(col("Value"))).orderBy(col("ObservationTime")).withColumn("Value", when(col("Value") == -1.0, 0).otherwise(col("Value")))
display(rain_observations)


StatementMeta(kipina, 10, 92, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 68bffa61-cfb5-4440-b9d7-30cd7c89dc43)

In [78]:
pyorat = bikes.withColumn("Date", to_date(col("Departure")))
pyorat_lampotilat = pyorat.join(temp_observations, pyorat.Date == temp_observations.ObservationDate).withColumnRenamed("Value", "Temp")
pyorat_lampotilat = pyorat_lampotilat.join(rain_observations, pyorat.Date == rain_observations.ObservationDate).withColumnRenamed("Value", "Rain")
pyorat_lampotilat.createOrReplaceTempView("pl")
display(pyorat_lampotilat)


StatementMeta(kipina, 10, 73, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4ff9cf6c-6d40-4668-b57e-92eaccf2760c)

In [79]:
paivat_matkat_temp = pyorat_lampotilat.groupBy(col("Date")).agg(count("*").alias("Trips"), avg(col("Temp")).alias("Temperature"), (sum(col("Covered distance (m)"))/1000).alias("Distance"), avg(col("Rain")).alias("Rain"))
display(paivat_matkat_temp)

StatementMeta(kipina, 10, 74, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9ac9a98f-b8b6-4736-8878-5fb1fb87a94e)

In [80]:
paivat_matkat_temp.toPandas().corr()

StatementMeta(kipina, 10, 75, Finished, Available, Finished)

/tmp/ipykernel_93059/1539100129.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  paivat_matkat_temp.toPandas().corr()


,Trips,Temperature,Distance,Rain
Trips,1.000000,0.561754,0.993591,-0.277395
Temperature,0.561754,1.000000,0.541640,0.162739
Distance,0.993591,0.541640,1.000000,-0.290401
Rain,-0.277395,0.162739,-0.290401,1.000000
